<a href="https://colab.research.google.com/github/chrlatte/TransferLearning/blob/main/Transfer_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install conplex-dti


In [ ]:
!conplex-dti download --to . --models ConPLex_v1_BindingDB

2023-07-12 13:38:59,158 [INFO] Download Location: /content
2023-07-12 13:38:59,158 [INFO] 
2023-07-12 13:38:59,158 [INFO] [BENCHMARKS]
2023-07-12 13:38:59,158 [INFO] [MODELS]
2023-07-12 13:38:59,158 [INFO] Downloading ConPLex_v1_BindingDB from https://cb.csail.mit.edu/cb/conplex/data/models/BindingDB_ExperimentalValidModel.pt to /content/models/ConPLex_v1_BindingDB.pt...


In [ ]:
model = nn.Module
# SimpleCoembeddingNoSigmoid(
#         drug_featurizer.shape, target_featurizer.shape, 1024
#     )
model.load_state_dict(torch.load(args.model_path, map_location=device))
model = model.eval()
model = model.to(device)


NameError: ignored